In [1]:
import torch
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import config
import os
from utils import (
    get_data_loaders,
    load_checkpoint,
    save_checkpoint,
    display_metrics,
    write_summary, 
    compute_metrics,
    get_TableMasks,
    fixMasks,
    display,
    display_predicted_and_fixed
    
    )
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
from model import TableNet
from pytorch_model_summary import summary
import tensorboard
from sklearn.model_selection import train_test_split
import albumentations as A 
from albumentations.pytorch import ToTensorV2
from PIL import Image, ImageDraw

C:\Users\user\anaconda3\envs\tablenet\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = TableNet(encoder = 'densenet', use_pretrained_model = True, basemodel_requires_grad = True)

print("Model Architecture and Trainable Paramerters")
print("="*50)
print(summary(model, torch.zeros((1, 3, 1024, 1024)), show_input=False, show_hierarchical=False))

C:\Users\user\anaconda3\envs\tablenet\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
C:\Users\user\anaconda3\envs\tablenet\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Model Architecture and Trainable Paramerters
-------------------------------------------------------------------------------------------------------------
      Layer (type)                                              Output Shape         Param #     Tr. Param #
        DenseNet-1     [1, 256, 64, 64], [1, 512, 32, 32], [1, 1024, 32, 32]       6,951,808       6,951,808
          Conv2d-2                                          [1, 256, 32, 32]         262,400         262,400
            ReLU-3                                          [1, 256, 32, 32]               0               0
         Dropout-4                                          [1, 256, 32, 32]               0               0
          Conv2d-5                                          [1, 256, 32, 32]          65,792          65,792
            ReLU-6                                          [1, 256, 32, 32]               0               0
         Dropout-7                                          [1, 256, 32, 32]      

In [4]:
model = TableNet(encoder = 'densenet', use_pretrained_model = True, basemodel_requires_grad = True)
model = model.to(config.DEVICE)


#load checkpoint
_,_,_ = load_checkpoint(torch.load("week_38/densenet_config_batch_size_4_model_checkpoint.pth.tar"), model)

Loading checkpoint...


In [6]:
processed_data = pd.read_csv(config.DATAPATH_TEST)
_, test_data  = train_test_split(processed_data, test_size = 0.2, random_state = config.SEED)

In [7]:
def createFixedMask(bboxes, dim):
    image = Image.new("RGB", dim)
    mask = ImageDraw.Draw(image)
    for each_list in bboxes:
        mask.rectangle(each_list, fill = 255)
    image_array = np.array(image)
    image = Image.fromarray(image_array[:,:,0])
    
    return image

In [12]:
for i in range(1):
    b = i + 1
#     print("Test image", test_data.iloc[a,0])
#     print("Test Image :", a)
    a = './resized_dilated/vendor/Compressor1_1st(0).png'
    test_img = np.array(Image.open(a))
    test_table = np.array(Image.open(a))
    table_out = get_TableMasks(test_img, model)
    table_out = table_out * 255
    print(table_out.shape)
    
#     backtorgb = cv2.cvtColor(table_out,cv2.COLOR_GRAY2RGB)
#     stacked_predicted_mask = np.stack([table_out, table_out, table_out], axis = 2)
#     print(stacked_predicted_mask)
    
    experiment_name =  str(b) + "-predicted_mask.png"

    cv2.imwrite(experiment_name, table_out)
#     cv2.imwrite('backtorgb.png', backtorgb)
#     predicted_mask_img = cv2.imread('predicted_mask.png')
#     print(predicted_mask_img)

#     display(test_img, test_table, table_out,title = 'Original')


    outputs = fixMasks(test_img, table_out)
    dim = (1024, 1024)
    fixed_mask = createFixedMask(outputs[1], dim)
   
    

    
    
#     display_predicted_and_fixed(test_img, table_out, fixed_mask)



# #     display(test_img, table_out, title = 'Test')


(1024, 1024, 1)


In [14]:
b = cv2.imread('mask.png')
cv2.imwrite('masked.png', b)

True

In [ ]:

#     fixed_img = cv2.imread('fixed_mask.png',0)
#     ret, bw_img = cv2.threshold(fixed_img,200,255,cv2.THRESH_BINARY)

    
#     original = 'orignal.png'
#     filename = 'bitwise.png'
    
    
#     print(test_img)
#     print(test_img.shape)

    
#     print(bw_img)
#     print(bw_img.shape)
    
#     bitwiseAnd = cv2.bitwise_and(bw_img, test_img)

#     cv2.imwrite(filename, bitwiseAnd)
#     cv2.imwrite(original, test_img)